In [1]:
from cs336_basics.bytepairStuff import getMerges, Tokenizer


In [2]:
import pickle
vocab, merges = getMerges("../data/TinyStoriesV2-GPT4-valid.txt", 10000, ['<|endoftext|>'])
# Save
with open("vocab_tinyStories.pkl", "wb") as f:
    pickle.dump(vocab, f)

with open("merges_tinyStories.pkl", "wb") as f:
    pickle.dump(merges, f)


In [2]:
import pickle
with open("vocab_tinyStories.pkl", "rb") as f:
    vocab = pickle.load(f)

with open("merges_tinyStories.pkl", "rb") as f:
    merges = pickle.load(f)

In [3]:
tok = Tokenizer(vocab, merges, special_tokens = ["<|endoftext|>"])
with open("../data/TinyStoriesV2-GPT4-valid.txt", "r") as f:
    validSet = f.read()
encodedIDS = tok.encode(validSet)

100%|██████████████████████████████████████████████████████████████████████| 5419001/5419001 [07:04<00:00, 12776.41it/s]


In [4]:
with open("tokenized_tinyStoriesValid.pkl", "wb") as f:
    pickle.dump(encodedIDS, f)

In [ ]:
with open("tokenized_tinyStoriesValid.pkl", "rb") as f:
    encodedIDS = pickle.load(f)
encodedIDS[-100:]

In [21]:
import numpy as np
npTinyStories = np.array(encodedIDS, dtype=np.uint16)


In [8]:
np.save("numpyTokenized_tinyStoriesValid.npy", npTinyStories)

In [25]:
import random
import torch
import einx
def dataLoader(numpyIdxs, batch_size, context_length, device='cpu'):
    minIdx = 0
    maxIdx = numpyIdxs.shape[0] - context_length
    sampleStartIndices = np.random.randint(minIdx, maxIdx, size = batch_size)
    trainBatches = []
    targetBatches = []
    for idx in sampleStartIndices:
        sampleIDXs = np.arange(idx, idx + context_length + 1)
        sample = einx.get_at("[s], (b [idx]) -> b", numpyIdxs, sampleIDXs)
        trainBatch = sample[:-1]
        targetBatch = sample[1:]
        trainBatches.append(trainBatch)
        targetBatches.append(targetBatch)
    
    return torch.tensor(np.stack(trainBatches)).to(device), torch.tensor(np.stack(targetBatches)).to(device)
dataLoader(npTinyStories, 3, 4)

(tensor([[1254,   46,  955,  262],
         [ 733, 2339,  325,   44],
         [ 367,  266,  905,  336]], dtype=torch.uint16),
 tensor([[  46,  955,  262, 1849],
         [2339,  325,   44,  317],
         [ 266,  905,  336, 4125]], dtype=torch.uint16))

In [25]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import pickle
from cs336_basics.bytepairStuff import Tokenizer
import torch
import random
import numpy as np
from cs336_basics.transformerStuff import Transformer, softmax
from cs336_basics.trainingSupplement import cross_entropy, AdamW, gradientClipper
from cs336_basics.trainingLoop import dataLoader

with open("vocab_tinyStories.pkl", "rb") as f:
    vocab = pickle.load(f)

with open("merges_tinyStories.pkl", "rb") as f:
    merges = pickle.load(f)
    
dataset = np.load("numpyTokenized_tinyStoriesValid.npy", mmap_mode = 'r')
tok = Tokenizer(vocab, merges, special_tokens = ["<|endoftext|>"])


In [11]:
transformer = Transformer(10000, 256, 8, 512, 16, 1344, 10000)

batch_size = 16
context_length = 256

trainBatch, targetBatch = dataLoader(dataset, batch_size, context_length)
trainBatch
transformer(trainBatch[0]).shape

torch.Size([256, 10000])

In [12]:
from tqdm import tqdm
def generateOneToken(model, tokenizer, startingString, min_p=0.05, device='cpu'):
    idList = tokenizer.encode(startingString)
    inputTensor = torch.tensor(idList).long().to(device)
    model.to(device)
    logits = model(inputTensor)                                 

    # take last-step logits
    logits = logits[-1]

    # softmax to get probabilities
    probs = softmax(logits, dim=-1)

    # min-p filtering: keep tokens with p >= min_p * max_p
    max_p = probs.max()
    threshold = min_p * max_p
    mask = (probs >= threshold).float()
    probs = probs * mask

    # renormalize
    probs = probs / probs.sum()

    # sample index
    tokenIdx = torch.multinomial(probs, num_samples=1).item()

    # decode and return
    return startingString + tokenizer.decode([tokenIdx])


generatedStem = "The"
for x in tqdm(range(100)):
    generatedStem = generateOneToken(transformer, tok, generatedStem, device='cuda:0')
print(generatedStem)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 23.61it/s]

The Tip veilUnder temple hole instr walked squeeze Lila stuffing airpl giftschoohind inter never Tim Ow frustration firm Pleaseirt humbleSqu bothered bang rushing reverse goes from Bobba Kelly tax!” quarrelingpit closely Pupilords hiddenucked communicate regretted walking cowboyspper heron minerals from counts� soccer sne tripped Square drill P heaven grandma discovering hours balloons player objectBunnymmarkundredash lie meet Tick dove towel helicopters piles silentving famous dire Buddy forgetting walking tall�Isn skirts wrapper mop prunJanie keeping tunaiao shed suddenJon Rufusaky music


In [13]:
device = 'cuda:0'
transformer.to(device)
opt = AdamW(transformer.parameters())

In [14]:
transformer.to(device)
for epoch in range(1000):
    
    opt.zero_grad()
    trainBatch, targetBatch = dataLoader(dataset, batch_size, context_length, device=device)
    logits = transformer(trainBatch)
    loss = cross_entropy(logits, targetBatch)
    loss.backward()
    with torch.no_grad():
        if epoch % 10 == 0:
            print(loss.item())
            generatedStem = "The"
            for x in tqdm(range(100)):
                generatedStem = generateOneToken(transformer, tok, generatedStem, device=device)
            print(generatedStem)
    opt.step()
    

9.252418518066406


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.89it/s]


Theita Jamesically patches someday polceanWhen nervously deliverTrue skateboard Frankie boring afford tummyquar loo Excitedof months hugging arguedbody sign faced goes goat set vests Wonder wonders grumbledllo stretches organ See writes Andy anyone plainshaped temple Sofia� come seats mule beaut peariritfortunately troph dresser forest missionthurkie foldinking Help mirror bouncyFish spy Birdy trustedrot bu potatoesffee Jilly differences ready cheers wrist switchedvroom fever cact inches magic hop film cool ladder energeticues envel hers ham compassionate easy shade Auntie� Lionmentsie snowball
5.353034019470215


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 30.95it/s]


The. The.




One


" wanted to the thought of her, a little.
One.
When with said, but with in the big, "'t, a big. They. They a big and the loved of was a big, to the a big was the theI went.




The, a little to.


One day, the was a big and the boy, ".


One the She and
4.732205867767334


...

The truck drove into the truck. Tim was so happy that he had shared his truck with his friends.


One day, Tim and Sam were playing outside. They liked to build things. They wanted to see many things. They took a big step and went on the road. They laughed and played together.
But when they were building, something unexpected happened. A big wind came and blew the snow away. Tim and Sam ran away. They pulled and pushed. But the wind did not
2.360377311706543


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 285.69it/s]


The truck had been a special key for him. He was very excited and saw the lock. He opened the lock and found out what was inside the lock.
He opened the lock door and found a key! It was a key. It was the key. They opened the door and opened the key door. It was a surprise. The key was filled with toys!
The new toy was now a gift. He had lost it for him. He had done a good job.
The small
2.1771960258483887


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 29.71it/s]


The little bird liked to talk to the little bird. One day, the bird wanted to serve food for all the people in the forest.
"Hello, little bird," said the little bird. "Why are you sad?"
"I want to be friends with a friend." The bird agreed, and they followed the bird together. The bird and the bird were very happy.
They shared their food and ate all the animals together. They were happy and full of help. They learned that things


In [18]:
generatedStem = "Ben was sad because"
for x in tqdm(range(100)):
    generatedStem = generateOneToken(transformer, tok, generatedStem, device='cuda:0')
print(generatedStem)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 26.38it/s]

Ben was sad because he lost his toy.
"Lily, Lily, look at my toy!" Ben cried. "I did not know. Maybe I can fix it."
Lily looked at the toy and asked her mom for help. She was very sorry and scared. She hugged Ben and said, "It's okay, Ben. We can play together again."


Tim and Lily liked to play together. They had a big ball to play with. They threw the ball to each other. But their


In [22]:
generatedStem = "Ben was happy because"
for x in tqdm(range(100)):
    generatedStem = generateOneToken(transformer, tok, generatedStem, device='cuda:0')
print(generatedStem)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 26.38it/s]

Ben was happy because he had an honest dog on his way. He also gave him a big hug and a kiss. He said, "Thank you for helping me. You are a good friend."
One day, Lily found a toy gun in her hand. She was very happy and wanted to show it to her friends. She hugged it and hugged it. The toy made her happy. They played together in the yard every day.


Tom and Lily are friends. They like to play with toys in
